In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_csv('data/labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
top10 = df.breed.value_counts()[:10]
df_top10 = df[df.breed.isin(top10.index.values)]
df_top10

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
57,0162b3e0144fb1d1ab82fbff3ace9938,shih-tzu
70,01b36cb1b80ab8c3a7d2b7128ad21bdc,samoyed
...,...,...
10195,ff54d45962b3123bb67052e8e29a60e7,basenji
10200,ff7d9c08091acc3b18b869951feeb013,maltese_dog
10204,ffa0055ec324829882186bae29491645,maltese_dog
10207,ffa4e1bf959425bad9228b04af40ac76,basenji


# Encoding breeds to integers

In [4]:
df_top10["breed_coded"] = df_top10.breed.astype('category').cat.codes

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Resizing images to (64, 64, 3) with train/test ration 9/1

In [28]:
import os
path_to_imgs = 'data/train'

batch_size = 32
img_height = 64
img_width = 64
dim = (img_width, img_height)


def func(id):
    img = cv2.imread(f'{path_to_imgs}/{id}.jpg', cv2.IMREAD_UNCHANGED)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, train_labels, test_labels = train_test_split(df_top10.id, df_top10.breed_coded, 
                                                              test_size=0.1, random_state=42,
                                                             stratify=df_top10.breed_coded)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the rele

In [30]:
train_images = np.array([func(id) for id in X_train])
test_images = np.array([func(id) for id in X_test])

# Encoding labels to match requirements of the loss function

In [31]:
y_binary_train = keras.utils.to_categorical(train_labels)
y_binary_test = keras.utils.to_categorical(test_labels)

# Developing the model

In [44]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(img_height, img_width, 3)),
    keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation='softmax')])

In [45]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

model.compile(optimizer=optimizer,
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 64, 64, 16)        448       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 32, 32, 64)        9280      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                163850    
Total params: 173,578
Trainable params: 173,578
Non-trainable params: 0
________________________________________________

In [47]:
EPOCHS = 20
history = model.fit(
    train_images, y_binary_train, epochs=EPOCHS
)

Train on 1026 samples
Epoch 1/20
1026/1026 [==============================] - 2s 2ms/sample - loss: 2.3676 - accuracy: 0.0936
Epoch 2/20
1026/1026 [==============================] - 2s 2ms/sample - loss: 2.3637 - accuracy: 0.0975
Epoch 3/20
1026/1026 [==============================] - 2s 2ms/sample - loss: 2.3637 - accuracy: 0.0975
Epoch 4/20
1026/1026 [==============================] - 2s 2ms/sample - loss: 2.3637 - accuracy: 0.0975
Epoch 5/20
1026/1026 [==============================] - 2s 2ms/sample - loss: 2.3637 - accuracy: 0.0975
Epoch 6/20
 448/1026 [============>.................] - ETA: 1s - loss: 2.3674 - accuracy: 0.0938

KeyboardInterrupt: 

In [ ]:
# Fails to converge